# Home Sales Notebook

In [3]:
#Cities in Snohomish
snohomish_cities = [
    'Arlington', 
    'Bothell',
    'Brier',
    'Darrington',
    'Edmonds',
    'Everett',
    'Gold Bar',
    'Granite Falls',
    'Index',
    'Lake Stevens',
    'Lynnwood',
    'Marysville',
    'Mill Creek',
    'Monroe',
    'Mountlake Terrace',
    'Mukilteo',
    'Snohomish',
    'Stanwood',
    'Sultan',
    'Woodway']

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [153]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
import pprint

import copy
import pandas as pd
import numpy as np

# Requests sends and recieves HTTP requests.
import requests
import time

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

from datetime import date

## create Mongo Database for scraped webpages

In [146]:
db = client['homesales_snohomish']
sales = db.sales

In [17]:
# get website
sale_search_url = 'http://www.snoco.org/app4/sas/assessor/services/salessearch2.aspx?TextBox1={}&TextBox12=&DropDownList3=&DropDownList1=&DropDownList2=&TextBox2=&TextBox3=&TextBox4=&TextBox5=&TextBox8=&TextBox9=&TextBox10={}%2F{}%2F{}&TextBox11={}%2F{}%2F{}'

def get_sales(city, start_date, end_date):
    # Return search URL with given parameters
    # Parameters:
    # city: str - city to search
    # start_date: date
    sale_search_url = 'http://www.snoco.org/app4/sas/assessor/services/salessearch2.aspx?TextBox1={city}&TextBox12=&DropDownList3=&DropDownList1=&DropDownList2=&TextBox2=&TextBox3=&TextBox4=&TextBox5=&TextBox8=&TextBox9=&TextBox10={s_m}%2F{s_d}%2F{s_y}&TextBox11={e_m}%2F{e_d}%2F{e_y}'
    search_term = sale_search_url.format(
        city='Arlington', 
        s_m=start_date.month, s_d=start_date.day, s_y=start_date.year,
        e_m=end_date.month, e_d=end_date.day, e_y=end_date.year)
    return search_term



In [26]:
print(get_sales(
    'Arlington', 
    date(2020, 1, 1), 
    date(2020, 1, 31)
))

http://www.snoco.org/app4/sas/assessor/services/salessearch2.aspx?TextBox1=Arlington&TextBox12=&DropDownList3=&DropDownList1=&DropDownList2=&TextBox2=&TextBox3=&TextBox4=&TextBox5=&TextBox8=&TextBox9=&TextBox10=1%2F1%2F2020&TextBox11=1%2F31%2F2020


In [18]:
r = requests.get(get_sales(
    'Arlington', 
    date(2020, 1, 1), 
    date(2020, 1, 31)
))


In [20]:
r.status_code

200

In [21]:
r.content

b'\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n\r\n<html dir="ltr" xmlns="http://www.w3.org/1999/xhtml">\r\n\r\n<head><meta content="text/html; charset=utf-8" http-equiv="Content-Type" /><title>\r\n\tSearch for Property Sales\r\n</title><script type="text/javascript">\r\nvar grd\r\ngrd = ""\r\nfunction fncaddgrid(x){\r\nvar k = document.getElementById(x).getElementsByTagName("td")\r\n//document.getElementById("rsltgrid").innerHTML+="<span>"\r\ngrd +="<td style=\'border-left:solid 1px black;\'>"\r\n//document.getElementById("rsltgrid").innerHTML+= k[0].innerHTML + "<br />"\r\ngrd+="<div style=\'height:30px;\'>" + k[0].innerHTML + "</div>"\r\n//document.getElementById("rsltgrid").innerHTML+= k[0].innerHTML + "<br />"\r\ngrd+="<div style=\'height:30px;\'>" + k[8].innerHTML + "</div>"\r\n//document.getElementById("rsltgrid").innerHTML+= k[8].innerHTML + "<br />"\r\ngrd+="<div style=\'height:30px;\'>" + k[9]

In [22]:
soup = BeautifulSoup(r.content, "html")

In [24]:
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html dir="ltr" xmlns="http://www.w3.org/1999/xhtml">
<head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><title>
	Search for Property Sales
</title><script type="text/javascript">
var grd
grd = ""
function fncaddgrid(x){
var k = document.getElementById(x).getElementsByTagName("td")
//document.getElementById("rsltgrid").innerHTML+="<span>"
grd +="<td style='border-left:solid 1px black;'>"
//document.getElementById("rsltgrid").innerHTML+= k[0].innerHTML + "<br />"
grd+="<div style='height:30px;'>" + k[0].innerHTML + "</div>"
//document.getElementById("rsltgrid").innerHTML+= k[0].innerHTML + "<br />"
grd+="<div style='height:30px;'>" + k[8].innerHTML + "</div>"
//document.getElementById("rsltgrid").innerHTML+= k[8].innerHTML + "<br />"
grd+="<div style='height:30px;'>" + k[9].innerHTML + "</div>"
//document.getElemen

In [25]:
print(soup.title)

<title>
	Search for Property Sales
</title>


In [66]:
# select the table
table = soup.find("table", {"id": "GridView1"})

In [111]:
rows = table.find_all("tr", recursive=False)

In [104]:
# ****** DO NOT CHANGE ********
empty_row = {
    "Parcel #": None, 
    "Date of Sale": None, 
    "Sale Price": None, 
    "Lot Size": None,
    "Year Built": None,
    "Type": None,
    "Quality/Grade": None,
    "Sq.ft.": None,
    "Address": None,
    "City": None,
    "Nbhd": None,
    "Use Code": None
}

In [133]:
row = rows[1]
columns = row.find_all("td")
columns[1].a

<a href="https://www.snoco.org/proptax/search.aspx?parcel_number=31051100400500            " style="color:#000066;" target=" blank">31051100400500            </a>

In [136]:
# Create an entry 
new_row = copy.copy(empty_row)
row = rows[1]
columns = row.find_all("td")
new_row['Parcel #'] = columns[1].a.text.strip()
new_row['Date of Sale'] = columns[2].text.strip()
new_row['Sale Price'] = columns[3].text.strip()
new_row['Lot Size'] = columns[4].text.strip()
new_row['Year Built'] = columns[5].text.strip()
new_row['Type'] = columns[6].text.strip()
new_row['Quality/Grade'] = columns[7].text.strip()
new_row['Sq.ft.'] = columns[8].text.strip()
new_row['Address'] = columns[9].text.strip()
new_row['City'] = columns[10].text.strip()
new_row['Nbhd'] = columns[11].text.strip()
new_row['Use Code'] = columns[12].text.strip()


In [137]:
new_row

{'Parcel #': 31051100400500,
 'Date of Sale': '1/31/2020 12:00:00 AM',
 'Sale Price': '$2,500,000.00',
 'Lot Size': 5.44,
 'Year Built': '',
 'Type': '',
 'Quality/Grade': '',
 'Sq.ft.': '',
 'Address': '21015 STATE ROUTE 9 NE',
 'City': 'Arlington',
 'Nbhd': 'Nbhd:5203000',
 'Use Code': 521}

### Month iterator

In [140]:
# Create month iterator

months = []
from pandas.tseries.offsets import MonthEnd

for beg in pd.date_range('2020-01-01', '2020-12-1', freq='MS'):
    months.append((
        date.fromisoformat( beg.strftime("%Y-%m-%d") ), 
        date.fromisoformat( (beg + MonthEnd(1)).strftime("%Y-%m-%d")) ))
    

In [141]:
months

[(datetime.date(2020, 1, 1), datetime.date(2020, 1, 31)),
 (datetime.date(2020, 2, 1), datetime.date(2020, 2, 29)),
 (datetime.date(2020, 3, 1), datetime.date(2020, 3, 31)),
 (datetime.date(2020, 4, 1), datetime.date(2020, 4, 30)),
 (datetime.date(2020, 5, 1), datetime.date(2020, 5, 31)),
 (datetime.date(2020, 6, 1), datetime.date(2020, 6, 30)),
 (datetime.date(2020, 7, 1), datetime.date(2020, 7, 31)),
 (datetime.date(2020, 8, 1), datetime.date(2020, 8, 31)),
 (datetime.date(2020, 9, 1), datetime.date(2020, 9, 30)),
 (datetime.date(2020, 10, 1), datetime.date(2020, 10, 31)),
 (datetime.date(2020, 11, 1), datetime.date(2020, 11, 30)),
 (datetime.date(2020, 12, 1), datetime.date(2020, 12, 31))]

In [145]:
months[0][0].year

2020

## Loop through cities and date range

In [157]:

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

In [148]:
all_rows = []

In [150]:
for city in snohomish_cities:
    for month in months:
        # Get info for each city and month
    
#         r = requests.get(get_sales(city, month[0], month[1]))
        soup = BeautifulSoup(r.content, "html")
        
        # select the table
        table = soup.find("table", {"id": "GridView1"})
        rows = table.find_all("tr", recursive=False)
        
        # loop through entries on page
        for row in rows[1:]:
            new_row = copy.copy(empty_row)
            columns = row.find_all("td")
            new_row['Parcel #'] = columns[1].a.text.strip()
            new_row['Date of Sale'] = columns[2].text.strip()
            new_row['Sale Price'] = columns[3].text.strip()
            new_row['Lot Size'] = columns[4].text.strip()
            new_row['Year Built'] = columns[5].text.strip()
            new_row['Type'] = columns[6].text.strip()
            new_row['Quality/Grade'] = columns[7].text.strip()
            new_row['Sqft'] = columns[8].text.strip()
            new_row['Address'] = columns[9].text.strip()
            new_row['City'] = columns[10].text.strip()
            new_row['Nbhd'] = columns[11].text.strip()
            new_row['Use Code'] = columns[12].text.strip()
            all_rows.append(new_row)
        
        # timer between request
        time.sleep(20 + np.random.random()*20)

            

AttributeError: 'NoneType' object has no attribute 'find_all'

In [152]:
len(all_rows)

520

In [158]:
all_rows

[{'Parcel #': '31051100400500',
  'Date of Sale': '1/31/2020 12:00:00 AM',
  'Sale Price': '$2,500,000.00',
  'Lot Size': '5.44',
  'Year Built': '',
  'Type': '',
  'Quality/Grade': '',
  'Sq.ft.': '',
  'Address': '21015 STATE ROUTE 9 NE',
  'City': 'Arlington',
  'Nbhd': 'Nbhd:5203000',
  'Use Code': '521'},
 {'Parcel #': '00804500000100',
  'Date of Sale': '1/31/2020 12:00:00 AM',
  'Sale Price': '$416,000.00',
  'Lot Size': '0.29',
  'Year Built': '1993',
  'Type': 'Tri Level',
  'Quality/Grade': 'Avg',
  'Sq.ft.': '1430',
  'Address': '20911 66TH AVE NE',
  'City': 'Arlington',
  'Nbhd': 'Nbhd:2408000',
  'Use Code': '111'},
 {'Parcel #': '00960009607300',
  'Date of Sale': '1/31/2020 12:00:00 AM',
  'Sale Price': '$75,000.00',
  'Lot Size': '0',
  'Year Built': '1996',
  'Type': 'Dbl Wide',
  'Quality/Grade': 'V Good',
  'Sq.ft.': '1620',
  'Address': '20227 80TH AVE NE SPC 73',
  'City': 'Arlington',
  'Nbhd': 'Nbhd:2408906',
  'Use Code': '119'},
 {'Parcel #': '00874800005200'

In [161]:
rows_cp = all_rows.copy()

In [166]:
for row in rows_cp:
    row['Sqft'] = row.pop('Sq.ft.', '')
    
    sales.insert_one(row)

In [168]:
rows = sales.find()
home_sales = pd.DataFrame(list(rows))

In [170]:
home_sales.drop('_id', axis=1)

,Parcel #,Date of Sale,Sale Price,Lot Size,Year Built,Type,Quality/Grade,Address,City,Nbhd,Use Code,Sqft
0,31051100400500,1/31/2020 12:00:00 AM,"$2,500,000.00",5.44,,,,21015 STATE ROUTE 9 NE,Arlington,Nbhd:5203000,521,
1,00804500000100,1/31/2020 12:00:00 AM,"$416,000.00",0.29,1993,Tri Level,Avg,20911 66TH AVE NE,Arlington,Nbhd:2408000,111,1430
2,00960009607300,1/31/2020 12:00:00 AM,"$75,000.00",0,1996,Dbl Wide,V Good,20227 80TH AVE NE SPC 73,Arlington,Nbhd:2408906,119,1620
3,00874800005200,1/29/2020 12:00:00 AM,"$419,000.00",0.14,2001,1 1/2 Sty,Avg,18127 GREYWALLS DR,Arlington,Nbhd:2408000,111,1813
4,00946600000300,1/29/2020 12:00:00 AM,"$347,000.00",0.1166,2003,1 Sty,Avg-,18722 35TH AVE NE,,Nbhd:2408000,111,1348
...,...,...,...,...,...,...,...,...,...,...,...,...
515,32063200300100,3/12/2020 12:00:00 AM,"$475,000.00",1.18,1935,1 Sty,Avg,11923 240TH ST NE,Arlington,Nbhd:4610000,111,1216
516,31052100305500,3/4/2020 12:00:00 AM,"$1,400,000.00",0.16,,,,17319 SMOKEY POINT BLVD,Arlington,Nbhd:5204000,641,
517,00455400701100,3/3/2020 12:00:00 AM,"$712,000.00",0.21,,,,609 1ST AVE,Arlington,Nbhd:5203000,124,
518,00960000303000,3/2/2020 12:00:00 AM,$500.00,0,1984,Sgl Wide,Fair,6206 188TH ST NE SPC 30,Arlington,Nbhd:2408901,119,924
